There are many parameters involved in tuning a nerual network. Below is an attempt to achieve 99% accuracy on the mnist dataset.

In [1]:
import tensorflow as tf
import keras

# Import the dataset
from keras.datasets import mnist

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop, Adam

# Import the backend
from keras import backend as K

import numpy as np

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
y_train[0:50]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0,
       9, 1, 1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9,
       3, 9, 8, 5, 9, 3], dtype=uint8)

In [4]:
# Change shape 
# Note that our images are 28*28 pixels, so in reshaping to arrays we want
# 60,000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train = (x_train - np.mean(x_train)) / np.std(x_train)
x_test = (x_test - np.mean(x_test)) / np.std(x_test)

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


Experimenting with multiple parameters including activation, layer units, amount of layers, dropout, optimization, loss and more.

In [24]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(128, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))



# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_194 (Dense)            (None, 128)               100480    
_________________________________________________________________
dropout_185 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_195 (Dense)            (None, 64)                8256      
_________________________________________________________________
dropout_186 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_196 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_187 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_197 (Dense)            (None, 64)                4160      
__________

In [25]:
history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 9s - loss: 0.4844 - acc: 0.8508 - val_loss: 0.1668 - val_acc: 0.9523
Epoch 2/50
60000/60000 [==============================] - 6s - loss: 0.2061 - acc: 0.9451 - val_loss: 0.1362 - val_acc: 0.9619
Epoch 3/50
60000/60000 [==============================] - 5s - loss: 0.1610 - acc: 0.9568 - val_loss: 0.1063 - val_acc: 0.9710
Epoch 4/50
60000/60000 [==============================] - 6s - loss: 0.1327 - acc: 0.9648 - val_loss: 0.1073 - val_acc: 0.9690
Epoch 5/50
60000/60000 [==============================] - 8s - loss: 0.1177 - acc: 0.9686 - val_loss: 0.0972 - val_acc: 0.9714
Epoch 6/50
60000/60000 [==============================] - 8s - loss: 0.1040 - acc: 0.9722 - val_loss: 0.0980 - val_acc: 0.9745
Epoch 7/50
60000/60000 [==============================] - 6s - loss: 0.0970 - acc: 0.9726 - val_loss: 0.0911 - val_acc: 0.9750
Epoch 8/50
60000/60000 [==============================] - 5s 

After numerous attempts the highest accuracy on the test set I was able to achieve is 98.3%